# Quantify

In [1]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from loguru import logger
import spatialdata as sd

from plex_pipe.utils.config_loaders import load_analysis_settings
from plex_pipe.object_quantification.controller import QuantificationController

## Read in config

In [2]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_analysis_settings(config_path)

# Specifies if a table with a given name should be overwritten if it already exists. 
# If False, the pipeline will throw an error if it tries to overwrite an existing resource.
# If True, the pipeline will overwrite existing resources. Use with caution!
OVERWRITE_FLAG = True

2026-02-19 09:26:52.282 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:49 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Define the logger

In [4]:
log_file = config.log_dir_path / f"rois_quantification_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

## Define ROIs for processing

In [5]:
roi_dir = config.analysis_dir / 'rois'
path_list = [roi_dir / f for f in os.listdir(roi_dir)]
path_list.sort()
path_list

[WindowsPath('../examples/output/sample_analysis/rois/ROI_000.zarr'),
 WindowsPath('../examples/output/sample_analysis/rois/ROI_001.zarr')]

In [6]:
# # create a subset of rois (optional)
# path_list = path_list[:1]

## Setup quantifiers

In [7]:
# setup quantification controllers
quant_controller_list = [] 
qc_prefix = config.qc.prefix

for quant in config.quant:

    table_name = quant.name
    masks_keys = quant.masks
    mask_to_annotate = quant.layer_connection
    qc_to_table = quant.qc_to_table

    logger.info(f"Setting up quantification controller for '{table_name}' table with masks {masks_keys} and connection to '{mask_to_annotate}' mask")

    controller = QuantificationController(
        table_name=table_name,
        mask_keys=masks_keys,
        mask_to_annotate=mask_to_annotate,
        overwrite=OVERWRITE_FLAG,
        add_qc_masks = qc_to_table,
        qc_prefix = qc_prefix,
    )

    quant_controller_list.append(controller) 

2026-02-19 09:27:07.734 | INFO     | __main__:<module>:12 - Setting up quantification controller for 'instanseg_table' table with masks {'nucleus': 'instanseg_nucleus', 'cell': 'instanseg_cell', 'ring': 'ring', 'cyto': 'cytoplasm'} and connection to 'instanseg_cell' mask


## Run ROIs Quantification

In [8]:
for sd_path in path_list:
    
    # load data
    logger.info(f'Quantifying {sd_path.name}')
    sdata = sd.read_zarr(sd_path)

    # run quantification
    for controller in quant_controller_list:
        controller.run(sdata)

2026-02-19 09:27:13.368 | INFO     | __main__:<module>:4 - Quantifying ROI_000.zarr
2026-02-19 09:27:14.376 | INFO     | plex_pipe.object_quantification.controller:validate_sdata_as_input:119 - Channels not specified. Quantifying all (3) existing channels: ['CD45', 'DAPI', 'NaKATPase'].
2026-02-19 09:27:14.376 | INFO     | plex_pipe.object_quantification.controller:run:557 - Spatial Data object is valid and ready for quantification.
2026-02-19 09:27:16.149 | INFO     | plex_pipe.object_quantification.controller:build_obs:199 - Quantifying morphology features for mask 'nucleus'
2026-02-19 09:27:34.376 | INFO     | plex_pipe.object_quantification.controller:build_obs:199 - Quantifying morphology features for mask 'cell'
2026-02-19 09:27:57.779 | INFO     | plex_pipe.object_quantification.controller:build_obs:199 - Quantifying morphology features for mask 'ring'
2026-02-19 09:28:17.900 | INFO     | plex_pipe.object_quantification.controller:build_obs:199 - Quantifying morphology features 

D:\uv\python\cpython-3.12-windows-x86_64-none\Lib\functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
d:\plex-pipe\.venv\Lib\site-packages\spatialdata\models\models.py:1206: UserWarning: Converting `region_key: region` to categorical dtype.
  convert_region_column_to_categorical(adata)


2026-02-19 09:28:57.519 | INFO     | plex_pipe.object_quantification.controller:run:563 - Quantification complete. Resulting AnnData has 12451 observations and 25 variables.
2026-02-19 09:28:57.568 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:31 - Table instanseg_table present in the spatialdata object.
2026-02-19 09:28:57.568 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:40 - Centroids: centroid_cell present in the anndata table instanseg_table.
2026-02-19 09:28:57.653 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'label' (key: qc_exclude_label).
2026-02-19 09:28:57.654 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'CD45' (key: qc_exclude_CD45).
2026-02-19 09:28:57.654 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'DAPI' (key: qc_exc

D:\uv\python\cpython-3.12-windows-x86_64-none\Lib\functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)
d:\plex-pipe\.venv\Lib\site-packages\spatialdata\models\models.py:1206: UserWarning: Converting `region_key: region` to categorical dtype.
  convert_region_column_to_categorical(adata)


2026-02-19 09:31:01.029 | INFO     | plex_pipe.object_quantification.controller:run:563 - Quantification complete. Resulting AnnData has 12167 observations and 25 variables.
2026-02-19 09:31:01.062 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:31 - Table instanseg_table present in the spatialdata object.
2026-02-19 09:31:01.062 | INFO     | plex_pipe.object_quantification.qc_shape_masker:validate_sdata:40 - Centroids: centroid_cell present in the anndata table instanseg_table.
2026-02-19 09:31:01.210 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'label' (key: qc_exclude_label).
2026-02-19 09:31:01.210 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'CD45' (key: qc_exclude_CD45).
2026-02-19 09:31:01.210 | DEBUG    | plex_pipe.object_quantification.qc_shape_masker:build_qc_mask:117 - No QC shapes found for marker 'DAPI' (key: qc_exc